In [1]:
import pandas as pd
from os.path import join, isfile
from os import listdir
import numpy as np

In [2]:
datafiles = [join('.\\data', f) for f in listdir('./data') if join('./data', f).endswith(".csv")]

In [3]:
datafiles

['.\\data\\Guess_2020_part0_2025-05-14_17h56.07.624.csv',
 '.\\data\\Guess_2020_part1_part2_2025-05-14_18h17.02.827.csv',
 '.\\data\\Guess_2020_sm_2025-05-14_18h51.28.478.csv',
 '.\\data\\Guess_2345_part0_2025-05-12_12h52.06.678.csv',
 '.\\data\\Guess_2345_part1_part2_2025-05-12_13h19.59.945.csv',
 '.\\data\\Guess_2345_sm_2025-05-12_13h58.25.708.csv',
 '.\\data\\Guess_3415_part0_2025-05-07_18h06.43.880.csv',
 '.\\data\\Guess_3415_part1_part2_2025-05-07_18h18.40.789.csv',
 '.\\data\\Guess_3415_sm_2025-05-07_19h02.04.177.csv',
 '.\\data\\Guess_4626_part1_part2_2025-05-21_18h10.12.067.csv',
 '.\\data\\Guess_4626_sm_2025-05-21_18h49.54.569.csv',
 '.\\data\\Guess_5579_part0_2025-05-10_12h20.15.733.csv',
 '.\\data\\Guess_5579_part1_part2_2025-05-10_12h33.23.619.csv',
 '.\\data\\Guess_5579_sm_2025-05-10_13h11.39.088.csv',
 '.\\data\\Guess_7328_part0_2025-05-10_14h29.28.579.csv',
 '.\\data\\Guess_7328_part1_part2_2025-05-10_14h40.36.507.csv',
 '.\\data\\Guess_7328_sm_2025-05-10_15h12.32.724.cs

In [8]:
counter = 0
learning_frames = []
recall_frames = []

for f in datafiles:
    print(f)
    if "end_part1_text.started" in pd.read_csv(f):
        df = pd.read_csv(f, converters={'PID': str})
             
        df["subj_recalled"] = 0 # column in learning for when participants responded with recall
        df["corr_recalled"] = 0 # column for when participants correctly chose the last letter

        for row in df.loc[df["test_trials.thisIndex"] >= 0].iterrows():
            subj_recall = row[1]["recall_reached.keys"]
            corr_recall = row[1]["correct_choice"]
            cue =  row[1]["Cue"]
            if subj_recall == 3:
                df.loc[df["Cue"] == cue, "subj_recalled"] = 1
            else:
                df.loc[df["Cue"] == cue, "subj_recalled"] = 0
            
            if corr_recall == 1:
                df.loc[df["Cue"] == cue, "corr_recalled"] = 1
            else:
                df.loc[df["Cue"] == cue, "corr_recalled"] = 0

        # LEARNING
        learning = df.loc[df["learning_trials.thisN"] >= 0]
        learning_frames.append(learning)

        
        # RECALL
        recall = df.loc[df["test_trials.thisN"] >= 0]
        recall_frames.append(recall)
        counter += 1
print('Number of finished experiments', counter)

.\data\Guess_2020_part0_2025-05-14_17h56.07.624.csv
.\data\Guess_2020_part1_part2_2025-05-14_18h17.02.827.csv
.\data\Guess_2020_sm_2025-05-14_18h51.28.478.csv
.\data\Guess_2345_part0_2025-05-12_12h52.06.678.csv
.\data\Guess_2345_part1_part2_2025-05-12_13h19.59.945.csv
.\data\Guess_2345_sm_2025-05-12_13h58.25.708.csv
.\data\Guess_3415_part0_2025-05-07_18h06.43.880.csv
.\data\Guess_3415_part1_part2_2025-05-07_18h18.40.789.csv
.\data\Guess_3415_sm_2025-05-07_19h02.04.177.csv
.\data\Guess_4626_part1_part2_2025-05-21_18h10.12.067.csv
.\data\Guess_4626_sm_2025-05-21_18h49.54.569.csv
.\data\Guess_5579_part0_2025-05-10_12h20.15.733.csv
.\data\Guess_5579_part1_part2_2025-05-10_12h33.23.619.csv
.\data\Guess_5579_sm_2025-05-10_13h11.39.088.csv
.\data\Guess_7328_part0_2025-05-10_14h29.28.579.csv
.\data\Guess_7328_part1_part2_2025-05-10_14h40.36.507.csv
.\data\Guess_7328_sm_2025-05-10_15h12.32.724.csv
.\data\Guess_8294_part0_2025-05-10_10h25.12.507.csv
.\data\Guess_8294_part1_part2_2025-05-10_10h40

In [10]:
learning_frames[0]["corr_recalled"]

11    0
12    1
13    1
14    1
15    1
     ..
86    0
87    1
88    1
89    0
90    1
Name: corr_recalled, Length: 80, dtype: int64

In [12]:
delim = ' '
form = '%.5f'
# guess_dur = 3 # none in final version
guess_react_dur = 3
encoding_dur = 3

for frame in learning_frames:
    PID = frame["PID"].values[0]
    triggers = frame["wait_for_trigger.stopped"][frame["wait_for_trigger.stopped"].notna()].values
    print(PID)
    print(triggers)
    guess_reached_times1 = frame.loc[frame["trial_type"]=="Guess","guess_response.started"] - triggers[0]
    guess_reached_times2 = frame.loc[frame["trial_type"]=="Guess","guess_response.started"] - triggers[1]
    guess_reached_times1 = guess_reached_times1.values[guess_reached_times2.values < 0]
    guess_reached_times2 = guess_reached_times2.values[guess_reached_times2.values >= 0]
    guess_reached_times1 = np.array([guess_reached_times1, np.ones(len(guess_reached_times1)) * guess_react_dur, np.ones(len(guess_reached_times1))]).T
    guess_reached_times2 = np.array([guess_reached_times2, np.ones(len(guess_reached_times2)) * guess_react_dur, np.ones(len(guess_reached_times2))]).T
    np.savetxt('onset_files/' + PID + '_guess_response_onset_run1.txt', guess_reached_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_guess_response_onset_run2.txt', guess_reached_times2, fmt=form, delimiter=delim)

    # Guess VS Read
    encode_guess_times1 = frame.loc[frame["trial_type"]=="Guess","encode.started"] - triggers[0]
    encode_guess_times2 = frame.loc[frame["trial_type"]=="Guess","encode.started"] - triggers[1]
    encode_guess_times1 = encode_guess_times1.values[encode_guess_times2.values < 0]
    encode_guess_times2 = encode_guess_times2.values[encode_guess_times2.values >= 0]
    encode_guess_times1 = np.array([encode_guess_times1, np.ones(len(encode_guess_times1)) * encoding_dur, np.ones(len(encode_guess_times1))]).T
    encode_guess_times2 = np.array([encode_guess_times2, np.ones(len(encode_guess_times2)) * encoding_dur, np.ones(len(encode_guess_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_guess_onset_run1.txt', encode_guess_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_guess_onset_run2.txt', encode_guess_times2, fmt=form, delimiter=delim)

    encode_read_times1 = frame.loc[frame["trial_type"]=="Read","encode.started"] - triggers[0]
    encode_read_times2 = frame.loc[frame["trial_type"]=="Read","encode.started"] - triggers[1]
    encode_read_times1 = encode_read_times1.values[encode_read_times2.values < 0]
    encode_read_times2 = encode_read_times2.values[encode_read_times2.values >= 0]
    encode_read_times1 = np.array([encode_read_times1, np.ones(len(encode_read_times1)) * encoding_dur, np.ones(len(encode_read_times1))]).T
    encode_read_times2 = np.array([encode_read_times2, np.ones(len(encode_read_times2)) * encoding_dur, np.ones(len(encode_read_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_read_onset_run1.txt', encode_read_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_read_onset_run2.txt', encode_read_times2, fmt=form, delimiter=delim)

    # Subsequent memory
    # Guess
    encode_guess_correct_times1 = frame.loc[((frame["trial_type"]=="Guess") & (frame["corr_recalled"]==1)),"encode.started"] - triggers[0]
    encode_guess_correct_times2 = frame.loc[((frame["trial_type"]=="Guess") & (frame["corr_recalled"]==1)),"encode.started"] - triggers[1]
    encode_guess_correct_times1 = encode_guess_correct_times1.values[encode_guess_correct_times2.values < 0]
    encode_guess_correct_times2 = encode_guess_correct_times2.values[encode_guess_correct_times2.values >= 0]
    encode_guess_correct_times1 = np.array([encode_guess_correct_times1, np.ones(len(encode_guess_correct_times1)) * encoding_dur, np.ones(len(encode_guess_correct_times1))]).T
    encode_guess_correct_times2 = np.array([encode_guess_correct_times2, np.ones(len(encode_guess_correct_times2)) * encoding_dur, np.ones(len(encode_guess_correct_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_guess_correct_onset_run1.txt', encode_guess_correct_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_guess_correct_onset_run2.txt', encode_guess_correct_times2, fmt=form, delimiter=delim)

    encode_guess_incorrect_times1 = frame.loc[((frame["trial_type"]=="Guess") & (frame["corr_recalled"]!=1)),"encode.started"] - triggers[0]
    encode_guess_incorrect_times2 = frame.loc[((frame["trial_type"]=="Guess") & (frame["corr_recalled"]!=1)),"encode.started"] - triggers[1]
    encode_guess_incorrect_times1 = encode_guess_incorrect_times1.values[encode_guess_incorrect_times2.values < 0]
    encode_guess_incorrect_times2 = encode_guess_incorrect_times2.values[encode_guess_incorrect_times2.values >= 0]
    encode_guess_incorrect_times1 = np.array([encode_guess_incorrect_times1, np.ones(len(encode_guess_incorrect_times1)) * encoding_dur, np.ones(len(encode_guess_incorrect_times1))]).T
    encode_guess_incorrect_times2 = np.array([encode_guess_incorrect_times2, np.ones(len(encode_guess_incorrect_times2)) * encoding_dur, np.ones(len(encode_guess_incorrect_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_guess_incorrect_onset_run1.txt', encode_guess_incorrect_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_guess_incorrect_onset_run2.txt', encode_guess_incorrect_times2, fmt=form, delimiter=delim)

    # Read
    encode_read_correct_times1 = frame.loc[((frame["trial_type"]=="Read") & (frame["corr_recalled"]==1)),"encode.started"] - triggers[0]
    encode_read_correct_times2 = frame.loc[((frame["trial_type"]=="Read") & (frame["corr_recalled"]==1)),"encode.started"] - triggers[1]
    encode_read_correct_times1 = encode_read_correct_times1.values[encode_read_correct_times2.values < 0]
    encode_read_correct_times2 = encode_read_correct_times2.values[encode_read_correct_times2.values >= 0]
    encode_read_correct_times1 = np.array([encode_read_correct_times1, np.ones(len(encode_read_correct_times1)) * encoding_dur, np.ones(len(encode_read_correct_times1))]).T
    encode_read_correct_times2 = np.array([encode_read_correct_times2, np.ones(len(encode_read_correct_times2)) * encoding_dur, np.ones(len(encode_read_correct_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_read_correct_onset_run1.txt', encode_read_correct_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_read_correct_onset_run2.txt', encode_read_correct_times2, fmt=form, delimiter=delim)

    encode_read_incorrect_times1 = frame.loc[((frame["trial_type"]=="Read") & (frame["corr_recalled"]!=1)),"encode.started"] - triggers[0]
    encode_read_incorrect_times2 = frame.loc[((frame["trial_type"]=="Read") & (frame["corr_recalled"]!=1)),"encode.started"] - triggers[1]
    encode_read_incorrect_times1 = encode_read_incorrect_times1.values[encode_read_incorrect_times2.values < 0]
    encode_read_incorrect_times2 = encode_read_incorrect_times2.values[encode_read_incorrect_times2.values >= 0]
    encode_read_incorrect_times1 = np.array([encode_read_incorrect_times1, np.ones(len(encode_read_incorrect_times1)) * encoding_dur, np.ones(len(encode_read_incorrect_times1))]).T
    encode_read_incorrect_times2 = np.array([encode_read_incorrect_times2, np.ones(len(encode_read_incorrect_times2)) * encoding_dur, np.ones(len(encode_read_incorrect_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_read_incorrect_onset_run1.txt', encode_read_incorrect_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_read_incorrect_onset_run2.txt', encode_read_incorrect_times2, fmt=form, delimiter=delim)

2020
[178.4131492 582.1551271]
2345
[140.0220167 551.7831636]
Guess_3415
[140.6531095 560.5122765]
4626
[105.2354594 510.5116326]
Guess_5579
[117.6484196 532.4723313]
Guess_7328
[110.9273632 519.7518062]
Guess_8294
[149.5992789 560.4090944]
9378
[123.8052402 530.2148622]
BP001
[205.726821  648.2874646]
BP002
[166.317641  585.5682817]


In [7]:
delim = ' '
form = '%.5f'
# ATTENTION. These are different for Guess_2020 and Guess_2345
recall_dur = 3
recall_resp_dur = 1.5
recall_sel_dur = 2.5

for frame in recall_frames:
    PID = frame["PID"].values[0]
    triggers = frame["wait_for_trigger.stopped"][frame["wait_for_trigger.stopped"].notna()].values
    print(PID)
    print(triggers)

    if PID != "2020":
    # recall attempt - correct
    ## Guess
        recall_correct_guess1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1),"cued_recall.started"] - triggers[0]
        recall_correct_guess2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1),"cued_recall.started"] - triggers[1]
        recall_correct_guess1 = recall_correct_guess1.values[recall_correct_guess2.values < 0]
        recall_correct_guess2 = recall_correct_guess2.values[recall_correct_guess2.values >= 0]
        recall_correct_guess1 = np.array([recall_correct_guess1, np.ones(len(recall_correct_guess1)) * recall_dur, np.ones(len(recall_correct_guess1))]).T
        recall_correct_guess2 = np.array([recall_correct_guess2, np.ones(len(recall_correct_guess2)) * recall_dur, np.ones(len(recall_correct_guess2))]).T
        np.savetxt('onset_files/' + PID + '_correct_recall_guess_onset_run1.txt', recall_correct_guess1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_correct_recall_guess_onset_run2.txt', recall_correct_guess2, fmt=form, delimiter=delim)

        ## Read
        recall_correct_read1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1),"cued_recall.started"] - triggers[0]
        recall_correct_read2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1),"cued_recall.started"] - triggers[1]
        recall_correct_read1 = recall_correct_read1.values[recall_correct_read2.values < 0]
        recall_correct_read2 = recall_correct_read2.values[recall_correct_read2.values >= 0]
        recall_correct_read1 = np.array([recall_correct_read1, np.ones(len(recall_correct_read1)) * recall_dur, np.ones(len(recall_correct_read1))]).T
        recall_correct_read2 = np.array([recall_correct_read2, np.ones(len(recall_correct_read2)) * recall_dur, np.ones(len(recall_correct_read2))]).T
        np.savetxt('onset_files/' + PID + '_correct_recall_read_onset_run1.txt', recall_correct_read1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_correct_recall_read_onset_run2.txt', recall_correct_read2, fmt=form, delimiter=delim)
        
    # recall attempt - incorrect

        recall_incorrect_guess1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0),"cued_recall.started"] - triggers[0]
        recall_incorrect_guess2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0),"cued_recall.started"] - triggers[1]
        recall_incorrect_guess1 = recall_incorrect_guess1.values[recall_incorrect_guess2.values < 0]
        recall_incorrect_guess2 = recall_incorrect_guess2.values[recall_incorrect_guess2.values >= 0]
        recall_incorrect_guess1 = np.array([recall_incorrect_guess1, np.ones(len(recall_incorrect_guess1)) * recall_dur, np.ones(len(recall_incorrect_guess1))]).T
        recall_incorrect_guess2 = np.array([recall_incorrect_guess2, np.ones(len(recall_incorrect_guess2)) * recall_dur, np.ones(len(recall_incorrect_guess2))]).T
        np.savetxt('onset_files/' + PID + '_incorrect_recall_guess_onset_run1.txt', recall_incorrect_guess1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_incorrect_recall_guess_onset_run2.txt', recall_incorrect_guess2, fmt=form, delimiter=delim)

        recall_incorrect_read1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0),"cued_recall.started"] - triggers[0]
        recall_incorrect_read2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0),"cued_recall.started"] - triggers[1]
        recall_incorrect_read1 = recall_incorrect_read1.values[recall_incorrect_read2.values < 0]
        recall_incorrect_read2 = recall_incorrect_read2.values[recall_incorrect_read2.values >= 0]
        recall_incorrect_read1 = np.array([recall_incorrect_read1, np.ones(len(recall_incorrect_read1)) * recall_dur, np.ones(len(recall_incorrect_read1))]).T
        recall_incorrect_read2 = np.array([recall_incorrect_read2, np.ones(len(recall_incorrect_read2)) * recall_dur, np.ones(len(recall_incorrect_read2))]).T
        np.savetxt('onset_files/' + PID + '_incorrect_recall_read_onset_run1.txt', recall_incorrect_read1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_incorrect_recall_read_onset_run2.txt', recall_incorrect_read2, fmt=form, delimiter=delim)

    # recall response
        recall_response_guess_corr1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[0]
        recall_response_guess_corr2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[1]
        recall_response_guess_corr1 = recall_response_guess_corr1.values[recall_response_guess_corr2.values < 0]
        recall_response_guess_corr2 = recall_response_guess_corr2.values[recall_response_guess_corr2.values >= 0]
        recall_response_guess_corr1 = np.array([recall_response_guess_corr1, np.ones(len(recall_response_guess_corr1)) * recall_resp_dur, np.ones(len(recall_response_guess_corr1))]).T
        recall_response_guess_corr2 = np.array([recall_response_guess_corr2, np.ones(len(recall_response_guess_corr2)) * recall_resp_dur, np.ones(len(recall_response_guess_corr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_guess_corr_onset_run1.txt', recall_response_guess_corr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_guess_corr_onset_run2.txt', recall_response_guess_corr2, fmt=form, delimiter=delim)

        recall_response_guess_incorr1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[0]
        recall_response_guess_incorr2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[1]
        recall_response_guess_incorr1 = recall_response_guess_incorr1.values[recall_response_guess_incorr2.values < 0]
        recall_response_guess_incorr2 = recall_response_guess_incorr2.values[recall_response_guess_incorr2.values >= 0]
        recall_response_guess_incorr1 = np.array([recall_response_guess_incorr1, np.ones(len(recall_response_guess_incorr1)) * recall_resp_dur, np.ones(len(recall_response_guess_incorr1))]).T
        recall_response_guess_incorr2 = np.array([recall_response_guess_incorr2, np.ones(len(recall_response_guess_incorr2)) * recall_resp_dur, np.ones(len(recall_response_guess_incorr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_guess_incorr_onset_run1.txt', recall_response_guess_incorr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_guess_incorr_onset_run2.txt', recall_response_guess_incorr2, fmt=form, delimiter=delim)

        recall_response_read_corr1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[0]
        recall_response_read_corr2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[1]
        recall_response_read_corr1 = recall_response_read_corr1.values[recall_response_read_corr2.values < 0]
        recall_response_read_corr2 = recall_response_read_corr2.values[recall_response_read_corr2.values >= 0]
        recall_response_read_corr1 = np.array([recall_response_read_corr1, np.ones(len(recall_response_read_corr1)) * recall_resp_dur, np.ones(len(recall_response_read_corr1))]).T
        recall_response_read_corr2 = np.array([recall_response_read_corr2, np.ones(len(recall_response_read_corr2)) * recall_resp_dur, np.ones(len(recall_response_read_corr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_read_corr_onset_run1.txt', recall_response_read_corr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_read_corr_onset_run2.txt', recall_response_read_corr2, fmt=form, delimiter=delim)

        recall_response_read_incorr1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[0]
        recall_response_read_incorr2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[1]
        recall_response_read_incorr1 = recall_response_read_incorr1.values[recall_response_read_incorr2.values < 0]
        recall_response_read_incorr2 = recall_response_read_incorr2.values[recall_response_read_incorr2.values >= 0]
        recall_response_read_incorr1 = np.array([recall_response_read_incorr1, np.ones(len(recall_response_read_incorr1)) * recall_resp_dur, np.ones(len(recall_response_read_incorr1))]).T
        recall_response_read_incorr2 = np.array([recall_response_read_incorr2, np.ones(len(recall_response_read_incorr2)) * recall_resp_dur, np.ones(len(recall_response_read_incorr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_read_incorr_onset_run1.txt', recall_response_read_incorr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_read_incorr_onset_run2.txt', recall_response_read_incorr2, fmt=form, delimiter=delim)

        # recall selection
        recall_selection_corr1 = frame.loc[(frame["correct_choice"]==1), "recall_selection.started"] - triggers[0]
        recall_selection_corr2 = frame.loc[(frame["correct_choice"]==1), "recall_selection.started"] - triggers[1]
        recall_selection_corr1 = recall_selection_corr1.values[recall_selection_corr2.values < 0]
        recall_selection_corr2 = recall_selection_corr2.values[recall_selection_corr2.values >= 0]
        recall_selection_corr1 = np.array([recall_selection_corr1, np.ones(len(recall_selection_corr1)) * recall_sel_dur, np.ones(len(recall_selection_corr1))]).T
        recall_selection_corr2 = np.array([recall_selection_corr2, np.ones(len(recall_selection_corr2)) * recall_sel_dur, np.ones(len(recall_selection_corr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_selection_corr_onset_run1.txt', recall_selection_corr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_selection_corr_onset_run2.txt', recall_selection_corr2, fmt=form, delimiter=delim)

        recall_selection_incorr1 = frame.loc[(frame["correct_choice"]==0), "recall_selection.started"] - triggers[0]
        recall_selection_incorr2 = frame.loc[(frame["correct_choice"]==0), "recall_selection.started"] - triggers[1]
        recall_selection_incorr1 = recall_selection_incorr1.values[recall_selection_incorr2.values < 0]
        recall_selection_incorr2 = recall_selection_incorr2.values[recall_selection_incorr2.values >= 0]
        recall_selection_incorr1 = np.array([recall_selection_incorr1, np.ones(len(recall_selection_incorr1)) * recall_sel_dur, np.ones(len(recall_selection_incorr1))]).T
        recall_selection_incorr2 = np.array([recall_selection_incorr2, np.ones(len(recall_selection_incorr2)) * recall_sel_dur, np.ones(len(recall_selection_incorr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_selection_incorr_onset_run1.txt', recall_selection_incorr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_selection_incorr_onset_run2.txt', recall_selection_incorr2, fmt=form, delimiter=delim)
    
    else:
        continue

2020
[1057.0966663 1439.556588 ]
2345
[1035.2075541 1437.6856972]
Guess_3415
[1186.1247945 1657.2829507]
4626
[ 984.9502119 1430.3245269]


KeyError: 'cued_recall.started'

In [ ]:
delim = ' '
form = '%.5f'
# ATTENTION. These are different for Guess_2020 and Guess_2345
recall_resp_dur = 3
recall_sel_dur = 3

for frame in recall_frames:
    PID = frame["PID"].values[0]
    triggers = frame["wait_for_trigger.stopped"][frame["wait_for_trigger.stopped"].notna()].values
    print(PID)
    print(triggers)

    if PID == "2020":
    # recall response
        recall_response_guess_corr1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[0]
        recall_response_guess_corr2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[1]
        recall_response_guess_corr1 = recall_response_guess_corr1.values[recall_response_guess_corr2.values < 0]
        recall_response_guess_corr2 = recall_response_guess_corr2.values[recall_response_guess_corr2.values >= 0]
        recall_response_guess_corr1 = np.array([recall_response_guess_corr1, np.ones(len(recall_response_guess_corr1)) * recall_resp_dur, np.ones(len(recall_response_guess_corr1))]).T
        recall_response_guess_corr2 = np.array([recall_response_guess_corr2, np.ones(len(recall_response_guess_corr2)) * recall_resp_dur, np.ones(len(recall_response_guess_corr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_guess_corr_onset_run1.txt', recall_response_guess_corr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_guess_corr_onset_run2.txt', recall_response_guess_corr2, fmt=form, delimiter=delim)

        recall_response_guess_incorr1 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[0]
        recall_response_guess_incorr2 = frame.loc[(frame["trial_type"]=="Guess") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[1]
        recall_response_guess_incorr1 = recall_response_guess_incorr1.values[recall_response_guess_incorr2.values < 0]
        recall_response_guess_incorr2 = recall_response_guess_incorr2.values[recall_response_guess_incorr2.values >= 0]
        recall_response_guess_incorr1 = np.array([recall_response_guess_incorr1, np.ones(len(recall_response_guess_incorr1)) * recall_resp_dur, np.ones(len(recall_response_guess_incorr1))]).T
        recall_response_guess_incorr2 = np.array([recall_response_guess_incorr2, np.ones(len(recall_response_guess_incorr2)) * recall_resp_dur, np.ones(len(recall_response_guess_incorr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_guess_incorr_onset_run1.txt', recall_response_guess_incorr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_guess_incorr_onset_run2.txt', recall_response_guess_incorr2, fmt=form, delimiter=delim)

        recall_response_read_corr1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[0]
        recall_response_read_corr2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==1), "recall_response.started"] - triggers[1]
        recall_response_read_corr1 = recall_response_read_corr1.values[recall_response_read_corr2.values < 0]
        recall_response_read_corr2 = recall_response_read_corr2.values[recall_response_read_corr2.values >= 0]
        recall_response_read_corr1 = np.array([recall_response_read_corr1, np.ones(len(recall_response_read_corr1)) * recall_resp_dur, np.ones(len(recall_response_read_corr1))]).T
        recall_response_read_corr2 = np.array([recall_response_read_corr2, np.ones(len(recall_response_read_corr2)) * recall_resp_dur, np.ones(len(recall_response_read_corr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_read_corr_onset_run1.txt', recall_response_read_corr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_read_corr_onset_run2.txt', recall_response_read_corr2, fmt=form, delimiter=delim)

        recall_response_read_incorr1 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[0]
        recall_response_read_incorr2 = frame.loc[(frame["trial_type"]=="Read") & (frame["correct_choice"]==0), "recall_response.started"] - triggers[1]
        recall_response_read_incorr1 = recall_response_read_incorr1.values[recall_response_read_incorr2.values < 0]
        recall_response_read_incorr2 = recall_response_read_incorr2.values[recall_response_read_incorr2.values >= 0]
        recall_response_read_incorr1 = np.array([recall_response_read_incorr1, np.ones(len(recall_response_read_incorr1)) * recall_resp_dur, np.ones(len(recall_response_read_incorr1))]).T
        recall_response_read_incorr2 = np.array([recall_response_read_incorr2, np.ones(len(recall_response_read_incorr2)) * recall_resp_dur, np.ones(len(recall_response_read_incorr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_response_read_incorr_onset_run1.txt', recall_response_read_incorr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_response_read_incorr_onset_run2.txt', recall_response_read_incorr2, fmt=form, delimiter=delim)

        # recall selection
        recall_selection_corr1 = frame.loc[(frame["correct_choice"]==1), "recall_selection.started"] - triggers[0]
        recall_selection_corr2 = frame.loc[(frame["correct_choice"]==1), "recall_selection.started"] - triggers[1]
        recall_selection_corr1 = recall_selection_corr1.values[recall_selection_corr2.values < 0]
        recall_selection_corr2 = recall_selection_corr2.values[recall_selection_corr2.values >= 0]
        recall_selection_corr1 = np.array([recall_selection_corr1, np.ones(len(recall_selection_corr1)) * recall_sel_dur, np.ones(len(recall_selection_corr1))]).T
        recall_selection_corr2 = np.array([recall_selection_corr2, np.ones(len(recall_selection_corr2)) * recall_sel_dur, np.ones(len(recall_selection_corr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_selection_corr_onset_run1.txt', recall_selection_corr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_selection_corr_onset_run2.txt', recall_selection_corr2, fmt=form, delimiter=delim)

        recall_selection_incorr1 = frame.loc[(frame["correct_choice"]==0), "recall_selection.started"] - triggers[0]
        recall_selection_incorr2 = frame.loc[(frame["correct_choice"]==0), "recall_selection.started"] - triggers[1]
        recall_selection_incorr1 = recall_selection_incorr1.values[recall_selection_incorr2.values < 0]
        recall_selection_incorr2 = recall_selection_incorr2.values[recall_selection_incorr2.values >= 0]
        recall_selection_incorr1 = np.array([recall_selection_incorr1, np.ones(len(recall_selection_incorr1)) * recall_sel_dur, np.ones(len(recall_selection_incorr1))]).T
        recall_selection_incorr2 = np.array([recall_selection_incorr2, np.ones(len(recall_selection_incorr2)) * recall_sel_dur, np.ones(len(recall_selection_incorr2))]).T
        np.savetxt('onset_files/' + PID + '_recall_selection_incorr_onset_run1.txt', recall_selection_incorr1, fmt=form, delimiter=delim)
        np.savetxt('onset_files/' + PID + '_recall_selection_incorr_onset_run2.txt', recall_selection_incorr2, fmt=form, delimiter=delim)
    
    else:
        continue